# Data Merging

In [1]:
import pandas as pd
import janitor
import gc

In [8]:
sampled_cityName = ['Fresno',
 'San Diego',
 'Sacramento',
 'Riverside',
 'San Jose',
 'Santa Ana',
 'Anaheim',
 'Santa Rosa',
 'Merced',
 'Santa Clarita',
 'Alhambra',
 'Davis',
 'Montebello',
 'Burbank',
 'Huntington Park',
 'Bellflower',
 'Watsonville',
 'Gilroy',
 'Whittier',
 'Lynwood',
 'Lakewood',
 'Pico Rivera',
 'Lake Forest',
 'Livermore',
 'Chino Hills',
 'Paramount',
 'El Paso de Robles',
 'Pico Rivera',
 'Buena Park',
 'Whittier',
 'Calabasas',
 'Carpinteria',
 'Morro Bay',
 'San Carlos',
 'Solvang']

print(len(sampled_cityName))
print(len(set(sampled_cityName)))
print(set([x for x in sampled_cityName if sampled_cityName.count(x) > 1]))

35
33
{'Pico Rivera', 'Whittier'}


## 1. Demographic Data

1. Select only the columns required: city name ('Residence_Addresses_City'), unique voter id ('LALVOTERID'), voter's ethnicity ('EthnicGroups_EthnicGroup1Desc') and date when voter was registered ('Voters_OfficialRegDate')
2. Keep only the cities that were identified as being similar to RCV cities in CA (See ca_similarity_search.ipynb for reference) 
3. Keep only rows EthnicGroups_EthnicGroup1Desc == “European”,  “Likely African-American”,“Hispanic and Portuguese” and “East and South Asian” 
4. Keep only registered voters identified in 'Voters_OfficialRegDate'


In [6]:
# change the filepath as required
filepath = 'VM2--CA--2022-04-25/'

selected_variables = ['Residence_Addresses_City', 
                      'LALVOTERID',
                      'EthnicGroups_EthnicGroup1Desc',
                      'Voters_OfficialRegDate'
                     ]

state_demographic = pd.read_csv(f'{filepath}VM2--CA--2022-04-25-DEMOGRAPHIC.tab', 
                                sep='\t', dtype=str, encoding='unicode_escape',
                                usecols=selected_variables)


In [8]:
state_demographic.head(5)

,LALVOTERID,Residence_Addresses_City,EthnicGroups_EthnicGroup1Desc,Voters_OfficialRegDate
0,LALCA453164106,Oakland,Other,06/18/2021
1,LALCA453008306,Oakland,Likely African-American,04/01/2021
2,LALCA22129469,Oakland,European,11/16/2021
3,LALCA549803906,Oakland,Other,02/07/2022
4,LALCA24729024,San Leandro,European,02/28/2016


In [9]:
selected_ethnicities = ['European', 'Likely African-American','Hispanic and Portuguese', 'East and South Asian']

state_demographic_subset = state_demographic[state_demographic['Residence_Addresses_City'].isin(sampled_cityName) &
                                             state_demographic['EthnicGroups_EthnicGroup1Desc'].isin(selected_ethnicities) &
                                             state_demographic['Voters_OfficialRegDate'].notnull()
                                            ]
print(state_demographic_subset.shape)
state_demographic_subset.head()

(3110292, 4)


,LALVOTERID,Residence_Addresses_City,EthnicGroups_EthnicGroup1Desc,Voters_OfficialRegDate
6,LALCA22466723,Livermore,European,11/01/2021
7,LALCA22466636,Livermore,European,12/07/2021
8,LALCA22466642,Livermore,European,12/07/2021
115,LALCA581684072,Livermore,European,09/20/2019
116,LALCA549860099,Livermore,European,07/26/2020


In [ ]:
del state_demographic
gc.collect()

## 2. Vote History

1. Select only the columns that are 4 most recent General elections and 4 most recent Local_or_Municipal elections and EthnicGroups_EthnicGroup1Desc
2. Merge Vote History with the sampled Demographic Data 


In [10]:
# select only subset of rows to find the column names that are 4 most recent General and Local_or_Municipal elections
state_voterhistory = pd.read_csv(f'{filepath}VM2--CA--2022-04-25-VOTEHISTORY.tab',
                                 sep='\t', dtype=str, encoding='unicode_escape',
                                nrows=10)
                                
state_voterhistory.head(5)

,LALVOTERID,Special_2022_04_19,Special_2022_04_12,Special_2022_04_05,Special_2022_02_15,Special_2022_02_01,Special_2021_12_14,Special_2021_12_07,Special_2021_11_02,Consolidated_General_2021_11_02,...,BallotReturnDate_General_2018_11_06,BallotReturnDate_Primary_2018_06_05,BallotReturnDate_General_2016_11_08,BallotReturnDate_Primary_2016_06_07,BallotReturnDate_General_2014_11_04,BallotReturnDate_Primary_2014_06_03,BallotReturnDate_General_2012_11_06,BallotReturnDate_Primary_2012_06_05,BallotReturnDate_General_2010_11_02,BallotReturnDate_Primary_2010_06_08
0,LALCA453164106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11/07/2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LALCA453008306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LALCA22129469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11/06/2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LALCA549803906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LALCA24729024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
def get_4_recent_date(string, df):
    list_cols = [col for col in df.columns if col.startswith(string)]
    dates = [col.replace(string+'_', '') for col in list_cols]
    dates.sort(reverse=True)
    return [string+'_'+d for d in dates[:4]]

GE_cols = get_4_recent_date('General', state_voterhistory)
print(GE_cols)
LM_cols = get_4_recent_date('Local_or_Municipal', state_voterhistory)
print(LM_cols)

['General_2020_11_03', 'General_2018_11_06', 'General_2016_11_08', 'General_2014_11_04']
['Local_or_Municipal_2021_08_31', 'Local_or_Municipal_2021_07_20', 'Local_or_Municipal_2021_06_08', 'Local_or_Municipal_2021_06_01']


In [12]:
del state_voterhistory
gc.collect()

706

In [13]:
needed_variables = ['LALVOTERID'] + LM_cols + GE_cols

state_voterhistory = pd.read_csv(f'{filepath}VM2--CA--2022-04-25-VOTEHISTORY.tab',
                                 sep='\t', dtype=str, encoding='unicode_escape',
                                 usecols=needed_variables)
                                
state_voterhistory.head(5)

,LALVOTERID,Local_or_Municipal_2021_08_31,Local_or_Municipal_2021_07_20,Local_or_Municipal_2021_06_08,Local_or_Municipal_2021_06_01,General_2020_11_03,General_2018_11_06,General_2016_11_08,General_2014_11_04
0,LALCA453164106,NaN,NaN,NaN,NaN,Y,Y,Y,NaN
1,LALCA453008306,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
2,LALCA22129469,NaN,NaN,NaN,NaN,Y,Y,Y,Y
3,LALCA549803906,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN
4,LALCA24729024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
merged_file = pd.merge(state_voterhistory, state_demographic_subset,
                       how='inner', left_on='LALVOTERID', right_on='LALVOTERID')

merged_file.head(5)

,LALVOTERID,Local_or_Municipal_2021_08_31,Local_or_Municipal_2021_07_20,Local_or_Municipal_2021_06_08,Local_or_Municipal_2021_06_01,General_2020_11_03,General_2018_11_06,General_2016_11_08,General_2014_11_04,Residence_Addresses_City,EthnicGroups_EthnicGroup1Desc,Voters_OfficialRegDate
0,LALCA22466723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Livermore,European,11/01/2021
1,LALCA22466636,NaN,NaN,NaN,NaN,Y,Y,Y,Y,Livermore,European,12/07/2021
2,LALCA22466642,NaN,NaN,NaN,NaN,Y,Y,Y,Y,Livermore,European,12/07/2021
3,LALCA581684072,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,Livermore,European,09/20/2019
4,LALCA549860099,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,Livermore,European,07/26/2020


In [15]:
print(merged_file.shape)

(3110292, 12)


In [ ]:
merged_file = merged_file.reset_index(drop = False)

In [16]:
merged_file.to_csv('VM2--CA--2022-04-25-MERGED.csv', index=False)

# Calculate voter turnout using merged data

In [1]:
import pandas as pd
merged_file = pd.read_csv('VM2--CA--2022-04-25-MERGED.csv')

/var/folders/ww/21cvzb4164325blmx95x6cd80000gn/T/ipykernel_15414/3662557750.py:2: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_file = pd.read_csv('VM2--CA--2022-04-25-MERGED.csv')


In [2]:
merged_file.head()

,Unnamed: 0,LALVOTERID,Local_or_Municipal_2021_08_31,Local_or_Municipal_2021_07_20,Local_or_Municipal_2021_06_08,Local_or_Municipal_2021_06_01,General_2020_11_03,General_2018_11_06,General_2016_11_08,General_2014_11_04,Residence_Addresses_City,EthnicGroups_EthnicGroup1Desc,Voters_OfficialRegDate
0,0,LALCA22466723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Livermore,European,11/01/2021
1,1,LALCA22466636,NaN,NaN,NaN,NaN,Y,Y,Y,Y,Livermore,European,12/07/2021
2,2,LALCA22466642,NaN,NaN,NaN,NaN,Y,Y,Y,Y,Livermore,European,12/07/2021
3,3,LALCA581684072,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,Livermore,European,09/20/2019
4,4,LALCA549860099,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,Livermore,European,07/26/2020


In [3]:
def get_4_recent_date(string, df):
    list_cols = [col for col in df.columns if col.startswith(string)]
    dates = [col.replace(string+'_', '') for col in list_cols]
    dates.sort(reverse=True)
    return [string+'_'+d for d in dates[:4]]

GE_cols = get_4_recent_date('General', merged_file)
print(GE_cols)
LM_cols = get_4_recent_date('Local_or_Municipal', merged_file)
print(LM_cols)

['General_2020_11_03', 'General_2018_11_06', 'General_2016_11_08', 'General_2014_11_04']
['Local_or_Municipal_2021_08_31', 'Local_or_Municipal_2021_07_20', 'Local_or_Municipal_2021_06_08', 'Local_or_Municipal_2021_06_01']


In [4]:
merged_file[GE_cols+LM_cols] = merged_file[GE_cols+LM_cols].fillna('N')
merged_file.head()

,Unnamed: 0,LALVOTERID,Local_or_Municipal_2021_08_31,Local_or_Municipal_2021_07_20,Local_or_Municipal_2021_06_08,Local_or_Municipal_2021_06_01,General_2020_11_03,General_2018_11_06,General_2016_11_08,General_2014_11_04,Residence_Addresses_City,EthnicGroups_EthnicGroup1Desc,Voters_OfficialRegDate
0,0,LALCA22466723,N,N,N,N,N,N,N,N,Livermore,European,11/01/2021
1,1,LALCA22466636,N,N,N,N,Y,Y,Y,Y,Livermore,European,12/07/2021
2,2,LALCA22466642,N,N,N,N,Y,Y,Y,Y,Livermore,European,12/07/2021
3,3,LALCA581684072,N,N,N,N,Y,N,N,N,Livermore,European,09/20/2019
4,4,LALCA549860099,N,N,N,N,Y,N,N,N,Livermore,European,07/26/2020


In [5]:
list_ethnic_city = merged_file[['Residence_Addresses_City', 'EthnicGroups_EthnicGroup1Desc']].drop_duplicates()
list_ethnic_city_No = list_ethnic_city.copy()
list_ethnic_city_No['voted'] = 'N'
list_ethnic_city_Yes = list_ethnic_city.copy()
list_ethnic_city_Yes['voted'] = 'Y'
list_ethnic_city = pd.concat([list_ethnic_city_No, list_ethnic_city_Yes])

In [6]:
list_ethnic_city

,Residence_Addresses_City,EthnicGroups_EthnicGroup1Desc,voted
0,Livermore,European,N
10,Livermore,Hispanic and Portuguese,N
19,San Diego,European,N
20,San Diego,East and South Asian,N
23,San Diego,Hispanic and Portuguese,N
...,...,...,...
2968566,Santa Rosa,European,Y
2968569,Santa Rosa,Hispanic and Portuguese,Y
2968572,Santa Rosa,East and South Asian,Y
2968627,Santa Rosa,Likely African-American,Y


In [9]:
print(list_ethnic_city.Residence_Addresses_City.nunique())
for col in sampled_cityName:
    if col not in list_ethnic_city.Residence_Addresses_City.unique():
        print(col)

32
El Paso de Robles


In [10]:
elec_date_cols = GE_cols+LM_cols
for i in range(len(elec_date_cols)):
    col = elec_date_cols[i]
    voter_turnout_stats = merged_file.groupby(['Residence_Addresses_City', 'EthnicGroups_EthnicGroup1Desc', col]).size().agg(
      {'count_'+ col: lambda x: x, 'prop_'+ col:lambda x: x / x.sum(level=[0,1])}
      ).unstack(level=0).reset_index()
    voter_turnout_stats = voter_turnout_stats.rename(columns = {col: 'voted'})
    voter_turnout_stats = list_ethnic_city.merge(voter_turnout_stats, 
                                                 how = 'left',
                                                 on = ['Residence_Addresses_City', 'EthnicGroups_EthnicGroup1Desc', 'voted']) 
    if i == 0:
        voter_turnout_merge = voter_turnout_stats.copy() 
    else:
        voter_turnout_merge = voter_turnout_merge.merge(voter_turnout_stats, on = ['Residence_Addresses_City', 'EthnicGroups_EthnicGroup1Desc', 'voted'])

/var/folders/ww/21cvzb4164325blmx95x6cd80000gn/T/ipykernel_15414/3028302244.py:5: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  {'count_'+ col: lambda x: x, 'prop_'+ col:lambda x: x / x.sum(level=[0,1])}
/var/folders/ww/21cvzb4164325blmx95x6cd80000gn/T/ipykernel_15414/3028302244.py:5: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  {'count_'+ col: lambda x: x, 'prop_'+ col:lambda x: x / x.sum(level=[0,1])}
/var/folders/ww/21cvzb4164325blmx95x6cd80000gn/T/ipykernel_15414/3028302244.py:5: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.grou

In [11]:
total_city_ethnic = merged_file.groupby(['Residence_Addresses_City', 'EthnicGroups_EthnicGroup1Desc']).size().reset_index()
total_city_ethnic.columns = ['Residence_Addresses_City', 'EthnicGroups_EthnicGroup1Desc', 'total']
total_city_ethnic

,Residence_Addresses_City,EthnicGroups_EthnicGroup1Desc,total
0,Alhambra,East and South Asian,17451
1,Alhambra,European,7359
2,Alhambra,Hispanic and Portuguese,16596
3,Alhambra,Likely African-American,191
4,Anaheim,East and South Asian,26340
...,...,...,...
123,Watsonville,Likely African-American,42
124,Whittier,East and South Asian,3963
125,Whittier,European,26477
126,Whittier,Hispanic and Portuguese,76334


In [12]:
voter_turnout_merge = total_city_ethnic.merge(voter_turnout_merge, on = ['Residence_Addresses_City', 'EthnicGroups_EthnicGroup1Desc'])
voter_turnout_merge

,Residence_Addresses_City,EthnicGroups_EthnicGroup1Desc,total,voted,count_General_2020_11_03,prop_General_2020_11_03,count_General_2018_11_06,prop_General_2018_11_06,count_General_2016_11_08,prop_General_2016_11_08,count_General_2014_11_04,prop_General_2014_11_04,count_Local_or_Municipal_2021_08_31,prop_Local_or_Municipal_2021_08_31,count_Local_or_Municipal_2021_07_20,prop_Local_or_Municipal_2021_07_20,count_Local_or_Municipal_2021_06_08,prop_Local_or_Municipal_2021_06_08,count_Local_or_Municipal_2021_06_01,prop_Local_or_Municipal_2021_06_01
0,Alhambra,East and South Asian,17451,N,5316.0,0.304624,11161.0,0.639562,10410.0,0.596527,14489.0,0.830268,17451.0,1.0,17451.0,1.000000,17451.0,1.0,17451.0,1.000000
1,Alhambra,East and South Asian,17451,Y,12135.0,0.695376,6290.0,0.360438,7041.0,0.403473,2962.0,0.169732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alhambra,European,7359,N,1441.0,0.195815,3029.0,0.411605,2785.0,0.378448,5065.0,0.688273,7359.0,1.0,7359.0,1.000000,7359.0,1.0,7359.0,1.000000
3,Alhambra,European,7359,Y,5918.0,0.804185,4330.0,0.588395,4574.0,0.621552,2294.0,0.311727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alhambra,Hispanic and Portuguese,16596,N,3781.0,0.227826,8009.0,0.482586,6879.0,0.414497,13220.0,0.796577,16596.0,1.0,16596.0,1.000000,16596.0,1.0,16596.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,Whittier,European,26477,Y,21345.0,0.806171,15405.0,0.581826,16908.0,0.638592,8807.0,0.332628,NaN,NaN,25.0,0.000944,NaN,NaN,2.0,0.000076
252,Whittier,Hispanic and Portuguese,76334,N,19858.0,0.260146,41186.0,0.539550,34580.0,0.453009,63509.0,0.831988,76334.0,1.0,76302.0,0.999581,76334.0,1.0,76333.0,0.999987
253,Whittier,Hispanic and Portuguese,76334,Y,56476.0,0.739854,35148.0,0.460450,41754.0,0.546991,12825.0,0.168012,NaN,NaN,32.0,0.000419,NaN,NaN,1.0,0.000013
254,Whittier,Likely African-American,214,N,48.0,0.224299,106.0,0.495327,90.0,0.420561,166.0,0.775701,214.0,1.0,214.0,1.000000,214.0,1.0,214.0,1.000000


In [13]:
voter_turnout_merge = voter_turnout_merge.fillna(0)
voter_turnout_merge

,Residence_Addresses_City,EthnicGroups_EthnicGroup1Desc,total,voted,count_General_2020_11_03,prop_General_2020_11_03,count_General_2018_11_06,prop_General_2018_11_06,count_General_2016_11_08,prop_General_2016_11_08,count_General_2014_11_04,prop_General_2014_11_04,count_Local_or_Municipal_2021_08_31,prop_Local_or_Municipal_2021_08_31,count_Local_or_Municipal_2021_07_20,prop_Local_or_Municipal_2021_07_20,count_Local_or_Municipal_2021_06_08,prop_Local_or_Municipal_2021_06_08,count_Local_or_Municipal_2021_06_01,prop_Local_or_Municipal_2021_06_01
0,Alhambra,East and South Asian,17451,N,5316.0,0.304624,11161.0,0.639562,10410.0,0.596527,14489.0,0.830268,17451.0,1.0,17451.0,1.000000,17451.0,1.0,17451.0,1.000000
1,Alhambra,East and South Asian,17451,Y,12135.0,0.695376,6290.0,0.360438,7041.0,0.403473,2962.0,0.169732,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,Alhambra,European,7359,N,1441.0,0.195815,3029.0,0.411605,2785.0,0.378448,5065.0,0.688273,7359.0,1.0,7359.0,1.000000,7359.0,1.0,7359.0,1.000000
3,Alhambra,European,7359,Y,5918.0,0.804185,4330.0,0.588395,4574.0,0.621552,2294.0,0.311727,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,Alhambra,Hispanic and Portuguese,16596,N,3781.0,0.227826,8009.0,0.482586,6879.0,0.414497,13220.0,0.796577,16596.0,1.0,16596.0,1.000000,16596.0,1.0,16596.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,Whittier,European,26477,Y,21345.0,0.806171,15405.0,0.581826,16908.0,0.638592,8807.0,0.332628,0.0,0.0,25.0,0.000944,0.0,0.0,2.0,0.000076
252,Whittier,Hispanic and Portuguese,76334,N,19858.0,0.260146,41186.0,0.539550,34580.0,0.453009,63509.0,0.831988,76334.0,1.0,76302.0,0.999581,76334.0,1.0,76333.0,0.999987
253,Whittier,Hispanic and Portuguese,76334,Y,56476.0,0.739854,35148.0,0.460450,41754.0,0.546991,12825.0,0.168012,0.0,0.0,32.0,0.000419,0.0,0.0,1.0,0.000013
254,Whittier,Likely African-American,214,N,48.0,0.224299,106.0,0.495327,90.0,0.420561,166.0,0.775701,214.0,1.0,214.0,1.000000,214.0,1.0,214.0,1.000000


In [14]:
voter_turnout_merge = voter_turnout_merge[voter_turnout_merge['voted'] == 'Y']
voter_turnout_merge

,Residence_Addresses_City,EthnicGroups_EthnicGroup1Desc,total,voted,count_General_2020_11_03,prop_General_2020_11_03,count_General_2018_11_06,prop_General_2018_11_06,count_General_2016_11_08,prop_General_2016_11_08,count_General_2014_11_04,prop_General_2014_11_04,count_Local_or_Municipal_2021_08_31,prop_Local_or_Municipal_2021_08_31,count_Local_or_Municipal_2021_07_20,prop_Local_or_Municipal_2021_07_20,count_Local_or_Municipal_2021_06_08,prop_Local_or_Municipal_2021_06_08,count_Local_or_Municipal_2021_06_01,prop_Local_or_Municipal_2021_06_01
1,Alhambra,East and South Asian,17451,Y,12135.0,0.695376,6290.0,0.360438,7041.0,0.403473,2962.0,0.169732,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,Alhambra,European,7359,Y,5918.0,0.804185,4330.0,0.588395,4574.0,0.621552,2294.0,0.311727,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
5,Alhambra,Hispanic and Portuguese,16596,Y,12815.0,0.772174,8587.0,0.517414,9717.0,0.585503,3376.0,0.203423,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
7,Alhambra,Likely African-American,191,Y,139.0,0.727749,93.0,0.486911,90.0,0.471204,34.0,0.178010,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
9,Anaheim,East and South Asian,26340,Y,20542.0,0.779879,11191.0,0.424867,12245.0,0.464882,5986.0,0.227259,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,Watsonville,Likely African-American,42,Y,38.0,0.904762,29.0,0.690476,30.0,0.714286,16.0,0.380952,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
249,Whittier,East and South Asian,3963,Y,3020.0,0.762049,1702.0,0.429473,1911.0,0.482210,783.0,0.197578,0.0,0.0,1.0,0.000252,0.0,0.0,0.0,0.000000
251,Whittier,European,26477,Y,21345.0,0.806171,15405.0,0.581826,16908.0,0.638592,8807.0,0.332628,0.0,0.0,25.0,0.000944,0.0,0.0,2.0,0.000076
253,Whittier,Hispanic and Portuguese,76334,Y,56476.0,0.739854,35148.0,0.460450,41754.0,0.546991,12825.0,0.168012,0.0,0.0,32.0,0.000419,0.0,0.0,1.0,0.000013


In [15]:
voter_turnout_merge.to_csv('voter_turnout_stats', index=False)